In [23]:
import torch
import numpy as np
from typing import List
import scipy
from scipy import linalg
from sklearn.decomposition import PCA
import pickle
import tqdm
import random
torch.set_default_tensor_type(torch.FloatTensor)
from termcolor import colored
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print(color.BOLD + 'Hello World !' + color.END)


Hello World !


In [2]:
device = "cpu"

In [3]:
q = torch.randn(10)

In [4]:
with open("encodings.bert-base.250k.pickle", "rb") as f:
    data = pickle.load(f)
    
#data = [d for d in data if d["gold"].lower() not in ["is", "are", "have", "had", "has","then","or","and",",",
#                                                    ".","-","'","of","in","on","off","the","a","an", "be", "been", "from", "to",
#                                                    "about", "around", "near", "it", "he", "she", "her","him","their", "they", "i", "was", "were","them",
#                                                    "than"]]

data = data[:200000]

In [5]:
len(data)

200000

In [6]:
func_words = ["is", "are", "have", "had", "has","then","or","and",",",".","-","'","of","in","on","off","the","a","an", "be", "been", "from", "to",
"about", "around", "near", "it", "he", "she", "her","him","their", "they", "i", "was", "were","them","than"]

In [7]:
sents = np.array([d["sent"] for d in data])
masked_tokens = np.array([d["gold"] for d in data])
top_words = np.array([d["top_words"][0] for d in data])
masked_idx = np.array([d["masked_index"] for d in data])

In [8]:
pretty_print(sents[2], top_words[2], masked_idx[2])

NameError: name 'pretty_print' is not defined

In [9]:
with open("bert_embeddings.pickle", "rb") as f:
    W = pickle.load(f)
    
W =  torch.from_numpy(W).to(device)

In [10]:
H = np.array([d["vec_batch_norm"] for d in data])
with open("bert_large_embeddings.pickle", "rb") as f:
    W = pickle.load(f)
    
H,W = torch.from_numpy(H).to(device), torch.from_numpy(W).to(device)
H = H[:150000]
W = W.T
H.shape, W.shape
del data

In [11]:
W.device

device(type='cpu')

In [12]:
# N = 100000
# d = 128
# l = 25
# H = (torch.randn(N,d) + torch.rand(N,d)**2)

# H = H - torch.mean(H, dim = 0, keepdim = True)
# H = H
# W = torch.randn(d,l)
cov_H = torch.tensor(np.cov(H.detach().cpu().numpy(), rowvar = False))

In [13]:
cov_H = cov_H.float()

In [11]:
cov_H

tensor([[ 2.0543, -0.0862,  0.0393,  ...,  0.0104,  0.0823,  0.0120],
        [-0.0862,  2.0053,  0.1353,  ..., -0.0145, -0.1006, -0.2331],
        [ 0.0393,  0.1353,  1.8198,  ..., -0.0367, -0.1188, -0.0467],
        ...,
        [ 0.0104, -0.0145, -0.0367,  ...,  1.7584,  0.0947,  0.0909],
        [ 0.0823, -0.1006, -0.1188,  ...,  0.0947,  1.9568,  0.3486],
        [ 0.0120, -0.2331, -0.0467,  ...,  0.0909,  0.3486,  3.1694]])

In [ ]:
# import time
# start = time.time()
# Q = cov_H[:1000]@W
# print(Q)
# print(cov_H.shape, W.shape)
# print(time.time() - start)

In [ ]:
# u = torch.nn.Parameter(u)
# optimizer = torch.optim.SGD([u], lr=0.001, momentum=0.9, weight_decay=1e-4)

In [12]:
def get_cov_output_projected(u, cov_H, W):
    
    u_normed = u / torch.norm(u)
    P = torch.eye(cov_H.shape[0]) - (u_normed@u_normed.T)
    #P = u_normed@u_normed.T
    print(P.shape, W.shape, cov_H.shape, u_normed.shape)
    first = P@W
    print("done first")
    second = cov_H@first
    print("done second")
    third = P@second
    print("done third")
    fourth = W.T@third
    print("done fourth")
    return fourth
    #return W.T@P@cov_H@P@W

def get_cov_output_total(H,W,n=10000):
    with torch.no_grad():
        Y_hat = H[:n]@W 
        Y_hat = Y_hat - torch.mean(Y_hat, dim = 1, keepdim = True)
        return torch.sum(Y_hat*Y_hat)/Y_hat.shape[0]
    #return torch.tensor(np.cov(Y_hat.detach().cpu().numpy(), rowvar = False))

def eval_total_var(H,W):
    
    with torch.no_grad():
        k = 1000
        Y_sum = torch.zeros(W.shape[1]).to(device)
        Y_sqr_sum = torch.zeros(W.shape[1]).to(device)
    
        for i in range(0,len(H), k):
            Y = H[i:i+k]@W
            Y_sum += torch.sum(Y, dim = 0)
            Y_sqr_sum += torch.sum(Y**2, dim = 0)
    
        Y_sqr_sum /= len(H)
        Y_sum /= len(H)
        return (Y_sqr_sum - Y_sum**2).mean().detach().cpu().numpy().item() # mean over vocab
    
def get_loss_func(cov_output_projected):
    
    loss =  torch.sum(torch.diag(cov_output_projected))
    print("done loss calculation")
    return loss

def get_loss_func2(u, cov_H, W, k = 1000):

    if u.norm() > 1e-6:
        u_normed = u / torch.norm(u)
        #P = torch.eye(cov_H.shape[0]) - (u_normed@u_normed.T)
        #print(P.dtype, W.dtype, cov_H.dtype)
        P = u_normed@u_normed.T
    else:
        u_normed = u
        P = torch.zeros(u_normed.shape[0], u_normed.shape[0])
    first = P@W
    second = cov_H@first
    third = P@second
    fourth = -torch.sum(W*third)/third.shape[1] # mean over vocab
    #fourth = W*third/third.shape[1] # mean over vocab
    #fourth = torch.sum(fourth, dim = 0)
    #fourth, _ = torch.sort(fourth, descending = True)
    #return -torch.sum(fourth[:k])
    
    return fourth 

def get_projection_to_intersection_of_nullspaces(rowspace_projection_matrices: List[np.ndarray], input_dim: int):
    """
    Given a list of rowspace projection matrices P_R(w_1), ..., P_R(w_n),
    this function calculates the projection to the intersection of all nullspasces of the matrices w_1, ..., w_n.
    uses the intersection-projection formula of Ben-Israel 2013 http://benisrael.net/BEN-ISRAEL-NOV-30-13.pdf:
    N(w1)∩ N(w2) ∩ ... ∩ N(wn) = N(P_R(w1) + P_R(w2) + ... + P_R(wn))
    :param rowspace_projection_matrices: List[np.array], a list of rowspace projections
    :param dim: input dim
    """

    I = np.eye(input_dim)
    Q = np.sum(rowspace_projection_matrices, axis = 0)
    P = I - get_rowspace_projection(Q)

    return P

def get_rowspace_projection(W: np.ndarray) -> np.ndarray:
    """
    :param W: the matrix over its nullspace to project
    :return: the projection matrix over the rowspace
    """

    if np.allclose(W, 0):
        w_basis = np.zeros_like(W.T)
    else:
        w_basis = scipy.linalg.orth(W.T) # orthogonal basis

    P_W = w_basis.dot(w_basis.T) # orthogonal projection on W's rowspace

    return P_W

def get_first_pca(H):
    pca = PCA(n_components = 1)
    pca.fit(H)
    return torch.from_numpy(pca.components_.T)

def BCA(H,W,n_components, eps = 1e-8, max_iters = 1500, min_iters = 50, init_pca = True):
    
    P_nullspace = torch.eye(H.shape[1])
    results = []
    total_var_orig = eval_total_var(H,W) 
    remaining_var = total_var_orig
    print("Total var original: ", remaining_var)
    H_proj = H.clone()
    rowspace_projs = []
    
    for i in range(n_components):
        print("****" + "ITER {}".format(i) + "****")
        #H_proj = H@P_nullspace # remove previous component 
        H_proj = H
        cov_H = torch.from_numpy(np.cov(H_proj.detach().cpu().numpy(), rowvar = False)).float()
        
        if init_pca:
            u = get_first_pca(H_proj.detach().cpu().numpy())
        else:
            k = random.choice(range(1, len(H_proj)-1))
            #u = H_proj[k:k+1].T #torch.zeros(H_proj.shape[1], 1) #1e-2*torch.randn(H_proj.shape[1], 1)
            u = H_proj.mean(dim=0)
            u = u.unsqueeze(1)
            
        u = torch.nn.Parameter(u)
        optimizer = torch.optim.SGD([u], lr=1)
        #optimizer = torch.optim.Adam([u])
        
        diff = 10
        j = 0
        loss_vals = [np.inf]
        patience = 10
        patience_counter = 0
        
        while j < max_iters and patience_counter < patience:
            
            
            optimizer.zero_grad()
            loss = get_loss_func2(P_nullspace@u, cov_H, W)
            loss.backward()
            optimizer.step()
            loss_vals.append(loss.detach().cpu().numpy().item())
            diff = np.abs(loss_vals[-1] - loss_vals[-2])
            
            if diff > eps:
                patience_counter = 0
            else:
                if j > min_iters:
                    patience_counter += 1
                
            if j % 20 == 0: print("j, loss, ", j, loss.detach().cpu().numpy().item(), diff)
            j += 1
        print("finished after {} iters".format(j))
        
        # calculate new nullspace projection to neutralzie component u
        
        u_normed = u / torch.norm(u)
        rowspace_projs.append((u_normed@u_normed.T).detach().cpu().numpy())
        P_nullspace = torch.from_numpy(get_projection_to_intersection_of_nullspaces(rowspace_projs,cov_H.shape[0])).float()
        #P_nullspace = torch.eye(H_proj.shape[1]).double() - u_normed@u_normed.T
        
        # calcualte explained variance
        total_var = eval_total_var(H,W)
        total_var_projected = remaining_var - get_loss_func2(u,cov_H,W).detach().cpu().numpy().item()
        explained_var = total_var_projected / total_var_orig
        remaining_var = remaining_var - total_var_projected
        
        
        
        results.append({"vec": u_normed.squeeze().detach().cpu().numpy(), "projected_var": total_var_projected,
                       "total_var": total_var, "explained_var": explained_var*100})
        
        with open("bca.pickle", "wb") as f:
            pickle.dump(results, f)
            
    return results
        

In [15]:
bca = BCA(H,W,n_components=10, eps = 1e-4, init_pca = True)

Total var original:  5.432384490966797
****ITER 0****
j, loss,  0 -0.21525579690933228 inf
j, loss,  20 -10.433961868286133 5.7220458984375e-06
j, loss,  40 -10.43394660949707 9.5367431640625e-07
j, loss,  60 -10.433951377868652 9.5367431640625e-07
finished after 61 iters
****ITER 1****
j, loss,  0 -8.998700141906738 inf
j, loss,  20 -9.015023231506348 0.0
j, loss,  40 -9.015005111694336 9.5367431640625e-07
j, loss,  60 -9.015005111694336 1.9073486328125e-06
finished after 61 iters
****ITER 2****
j, loss,  0 -0.5915675163269043 inf
j, loss,  20 -0.11595319956541061 0.0012339204549789429
j, loss,  40 -0.14315110445022583 0.0014730393886566162
j, loss,  60 -0.17494933307170868 0.0016894787549972534
j, loss,  80 -0.2105572372674942 0.0018508434295654297
j, loss,  100 -0.24855534732341766 0.001929551362991333
j, loss,  120 -0.28710615634918213 0.001911073923110962


KeyboardInterrupt: 

In [ ]:
with open("bca-all-roberta.pickle", "wb") as f:
    pickle.dump(bca, f)

In [14]:
with open("bca-all.pickle", "rb") as f:
    bca = pickle.load(f)

In [15]:
len(bca)

50

In [ ]:
vecs = np.array([x["vec"] for x in bca])
for i,v in enumerate(vecs):
    for j, v2 in enumerate(vecs):
        if j <= i: continue
            
        print(i,j, v@v2.T)

In [ ]:
import scipy.optimize as optimize

def f(u,W,V, P_nullspace, orthogonal_constraints = []):
    norm = np.linalg.norm
    grad =  0.5 * ( (norm(u)**2/norm(V.T@u)**2)*V@V.T@u  + (norm(u)**2/norm(W.T@u)**2)*W@W.T@u)
#     c_orth = 10
#     for v in orthogonal_constraints:
#         t_0 = np.linalg.norm(u)
#         t_1 = (u).dot(v)
#         t_2 = (v).dot(u)
#         gradient = -((((2 / (t_0 ** 2)) * t_1) * v) - (((2 / (t_0 ** 4)) * t_2) * (t_2 * u)))        
#         print("gradient", np.linalg.norm(gradient), (u.T/(np.linalg.norm(u)))@v)
#         grad += -c_orth*gradient
        
    return grad

def f2(u,W,V,cov):
    
    S = cov
    t_0 = (u).dot(u)
    t_1 = (t_0 ** 4)
    t_2 = (W).dot((W.T).dot(u))
    t_3 = (u).dot(t_2)
    t_4 = (S).dot(u)
    t_5 = (1 / t_1)
    t_6 = (u).dot(t_4)
    t_7 = t_4
    t_8 = (4 / (t_0 ** 5))
    t_9 = (u).dot(t_7)
    t_10 = (t_3 * u)
    functionValue = ((t_6 * t_3) / t_1)
    gradient = ((((((t_5 * t_3) * t_4) + (t_5 * (t_3 * t_7))) + ((t_5 * t_6) * t_2)) + (t_5 * (t_9 * t_2))) - (((t_8 * t_6) * t_10) + ((t_8 * t_9) * t_10)))
    return gradient

def calculate_components(H,cov_H,W,n=2):
    comps = []
    H_proj = H.detach().cpu().numpy()
    cov = cov_H.detach().cpu().numpy()
    V = np.linalg.cholesky(cov)
    W = W.detach().cpu().numpy()
    P_nullspace = np.eye(768)
    u = 0.1*np.random.randn(768)
    for i in range(n):
        print("Iteration {}".format(i))
        print("=========================")
        u = P_nullspace@u
        u_last = u.copy()
        
        for j in range(200):
            u_last, u = u, f(u,W,V,P_nullspace,comps)
            diff = np.linalg.norm(u_last-u)
            if diff < 1e-5: break
            if j%10 == 0: print(diff)
                
        u_normed = u/np.linalg.norm(u)
        P_nullspace = P_nullspace@(np.eye(768) - u_normed@u_normed.T)
        comps.append(u_normed)

        print("Start test.")
        
        for k in range(len(comps)):
            for t in range(len(comps)):
                if k <= t: continue
                print("Test", comps[k].T@comps[t])
                
        print("Done test.")
        
        #P_nullspace = np.eye(768) - u@u.T
        #H_proj = H_proj@P_nullspace
        #cov = np.cov(H_proj, rowvar = False)
        #V = np.linalg.cholesky(cov)
    
    return comps 

comps = calculate_components(H,cov_H,W,n=2)

# u = np.random.randn(768,1)
# u_last = u.copy()

# i = 0
# while i < 100:
#     u_last, u = u, f(u)
#     print(np.linalg.norm(u_last-u))
#     i+=1

In [ ]:
u = bca[0]["vec"]
v = comps[0]
u.T@v

In [ ]:

#for i in range(len(bca)):
    
    #print(bca[i]["explained_var"])
    
#cov_out_total = get_cov_output_total(H,W)
#total_var = get_loss_func(cov_out_total).detach().cpu().numpy().item()    
vars = [x["explained_var"] for x in bca]
print(vars)
print(sum(vars))

In [ ]:
u = bca[5]["vec"]
pca = PCA(n_components = 25)
pca.fit(H)

In [ ]:
k=0
pca.components_[k].T@bca[k]["vec"]

In [ ]:
def pretty_print(sent, prediction, mask_index):
    sent_lst = sent.split(" ")
    sent_lst = sent_lst[:mask_index] + [colored("***{}***".format(sent_lst[mask_index]), "blue")] + [colored("(PRED: {})".format(prediction), "red")] + sent_lst[mask_index+1:]
    
    print (" ".join(sent_lst))

    
for i in range(50):
    print(color.BOLD + "*********************************" + "PRINCIPLE COMPONENT {}".format(i)+"*********************************\n\n" + color.END)
    u = bca[i]["vec"]


    #u = np.expand_dims(u, axis=1)
    vecs = H.detach().cpu().numpy()
    vecs_u = vecs@u
    idx = np.argsort(vecs_u)
    k = 250
    top_neg, top_pos = idx[:k], idx[-k:]
    filter_func = True

    if filter_func:
        mask_sents = np.array([True if top_words[i] not in func_words else False for i in range(len(sents))])
    else:
        mask_sents = np.ones_like(sents)
    
    sents_pos = sents[top_pos]
    sents_neg = sents[top_neg]
    gold_pos = masked_tokens[top_pos]
    gold_neg = masked_tokens[top_neg]
    preds_pos = top_words[top_pos]
    preds_neg = top_words[top_neg]
    idx_pos = masked_idx[top_pos]
    idx_neg = masked_idx[top_neg]

    for i in range(20):
    
        pretty_print(sents_pos[i], preds_pos[i], idx_pos[i])
        #print(gold_pos[i], preds_pos[i], " -------- ", sents_pos[i], "---------", vecs_u[top_pos[i]])
        print("======================================")

    print()
    print()
    print("**********************************************************")
    print("**********************************************************")
    print("**********************************************************\n\n\n")

    for i in range(20):
        pretty_print(sents_neg[i], preds_neg[i], idx_neg[i])
        print("======================================")

*********************************PRINCIPLE COMPONENT 0*********************************


The Treaty of Berlin ( 1889 ) was the concluding document ***of*** (PRED: of) the conference at Berlin in 1889 on Samoa .
The NPF was specifically chartered by Congress to further the conservation of natural , scenic , historic , scientific , educational , inspirational , or recreational resources ***in*** (PRED: within) the National Park System for future generations of Americans .
Work closely with the Municipal Council and the Antiquities Department to establish guidelines for conservation and development ***of*** (PRED: of) historic part of town .
Heredia was born in Mexico City and spent his life investigating and exposing the tricks ***of*** (PRED: of) spiritism .
Jianchangnathus is an extinct genus ***of*** (PRED: of) basal pterosaur from the Middle Jurassic Tiaojishan Formation of northeastern China .
The North Hill Historic District is a residential neighborhood in the city of New Castle 

She also appeared in regional theatre productions including Plaza Suite , It Had To be You , The Owl and the Pussycat , Goodbye ***Charlie*** (PRED: darling) and Cat on a Hot Tin Roof .
He also coached at Rindge and Latin , where his players included NBA players Patrick Ewing , Rumeal Robinson and former George Washington head coach Karl ***Hobbs*** (PRED: malone) .
Their course was down the Colorado River to the Mojave , where ***they*** (PRED: they) found two Indians , who guided them across the desert to San Gabriel Mission .
Also , Cochise by Audioslave is briefly heard on the radio while Homer is riding with ***Louie*** (PRED: bart) .
By the early 20th century , the Bacon movement faded , resulting in increased interest in Stanley and ***Oxford*** (PRED: bacon) .
On the January 5 , 2012 episode of Impact Wrestling , Kim and Rayne retained their tag team title against James and Traci ***Brooks*** (PRED: james) .
Among their successful publications was Bret Harte 's The Luck of Roar

In this ***second*** (PRED: brief) rebirth , the Indianapolis Police Department was able to survive and grow .
Astray is a tie-in to Stargazer , and expands upon ***the*** (PRED: the) technologies and organizations from this show .
Games , Life and Utopia , first published in 1978 , in which Suits defines the playing of a game as `` the voluntary attempt to overcome ***unnecessary*** (PRED: all) obstacles '' .
The sources are ok , and a reader can reasonably determine from ***the*** (PRED: these) differences that potential bias is at play .
The table below lists ***common*** (PRED: their) geometric shapes and a qualitative analysis of their two principal curvatures .
Because of the ***vague*** (PRED: unusual) nature of Palsgraf 's sole injury , some legal scholars have suggested that Cardozo reversed the jury 's verdict to erase a fraudulent claim .
Menus are more limited and standardized than in traditional pizzerias , and pizza delivery is ***offered*** (PRED: free) .
Army establishe

The resolution was approved by 14 votes to none against , while the United States ***abstained*** (PRED: ##tained) from voting .
Even though it was so hard to communicate with them in English , Ba ***was*** (PRED: was) a better speaker and knew many more words than the other men , so that he could translate for them .
Love was admitted to the University of Virginia , where she ***majored*** (PRED: ##ed) in government and minored in Spanish .
Rhys discovers the deaths and is appalled that Majere allowed his worshippers to be ***killed*** (PRED: killed) without lending aid .
With his football career over , Kiernan turned his attention to cricket and from 1910 to 1919 ***played*** (PRED: played) 10 first-class matches for Victoria , three of them in the Sheffield Shield .
From 2007-2009 , Khutsidze ***worked*** (PRED: worked) his way through the ranks at FC Ameri Tbilisi .
Of the 66 games played away from home , Zambia ***won*** (PRED: won) 28 times , drew 16 and lost 22 .
The Parti Québé

Western Wireless Corporation was a cellular network operator that provided mobile telecommunications service to subscribers in 19 western states and seven ***countries*** (PRED: territories) .
Lawyers Building from the state of Michigan The building originally catered to middle class level tenants , including many unions and benevolent ***organizations*** (PRED: societies) .
Azerbaijan has been since the ancient times known as a center of a large variety of ***crafts*** (PRED: cultures) .
The entire article is flawed and requires a rewrite , with a definition of what a metropolitan area is and reliable sources provided , such as information from ***statistics*** (PRED: unesco) .
One of these ***sources*** (PRED: issues) is `` The codification of general practice that already has wide consensus .
Some gifted and talented ***classes*** (PRED: schools) offer directed studies , where the students lead a class themselves and decide on their own projects , tests , and all other assignments .

In [ ]:
top_words[top_words not in func_words]

In [ ]:
idx = np.argsort(vecs_u)
vecs_u.shape

In [ ]:
np.linalg.norm(u)

In [ ]:
bca[-1]["cov_out"]

In [ ]:
bca[-3]["explained_var"]

In [ ]:
u,v = np.random.randn(768), np.random.randn(768).T
u,v = u / np.linalg.norm(u), v/np.linalg.norm(v)
u@v

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10,11,12]

In [ ]:
a[-12:-2]

In [ ]:
A = np.random.rand(768,32000)
H = np.random.rand(10000,768)
H*A

In [ ]:
W = torch.randn(768,32000)
H = torch.randn(10000, 768)
Y_hat = H[:10000]@W 
torch.mean(Y_hat*Y_hat)

In [ ]:
H.shape, W.shape

In [ ]:
H_first_comp = get_first_pca(H)
Y = H[:10000]@W
Y_first_comp = get_first_pca(Y)

In [ ]:
total_var = eval_total_var(H,W)
print(total_var)

In [ ]:
print(total_var.sum()/W.shape[1])

In [ ]:
u = torch.randn(768,1)
get_loss_func2(H_first_comp, cov_H, W)

In [ ]:
v = bca[0]["vec"]
v = np.expand_dims(v, 1)
v.shape

In [ ]:
get_loss_func2(torch.zeros(768,1), cov_H, W)

In [ ]:
cov_H.shape

In [ ]:
v.shape

In [ ]:
eval_total_var(H,W)

In [ ]:
q = torch.randn(10,3)

In [ ]:
import random
q[1:2].shape

In [ ]:
v = results[0]["vec"]

In [ ]:
np.linalg.norm(bca[5]["vec"])

In [22]:
colored("hah", "bold")

KeyError: 'bold'